## Opensoundscape API

Please give a short description of your model here and the benefit it provides.  Also, don't forget to mention your organization here so the world can know that this is your amazing work!

This template will load an image, show it, and call a fake API unrelated to that image.  We're guessing your API will do more interesting things.

### Imports and constants

In [1]:
import io
import json
import requests
 
%autosave 0

# WILL NEED TO GET UPDATED 
API_URL = "http://localhost:8081/v1/birds"

# WILL NEED TO GET UPDATED
SAMPLE_IMAGE_URL = "/home/bmooreii/projects/openbird/ai4e-example/client/cardinalis-cardinalis.wav"

Autosave disabled


### Functions

In [2]:
def get_audio():  
#     image_data = requests.get(SAMPLE_IMAGE_URL) 
#     return image_data.content
    with open(SAMPLE_IMAGE_URL, "rb") as audio:
        return audio.read()

    
def call_my_api(data):
    try:           
        r = requests.post(f"{API_URL}/detect", data=data)

        if r.ok:
            return r.json()
        else:
            return None

    except Exception as e:
        print('Exception running API: {}'.format(e))

### Retrieve and run prediction on audio

In [3]:
audio_bytes = get_audio()

### Call API and show results

In [4]:
# Get predictions
result = call_my_api(audio_bytes)
result

{'predictions': [['passer-montanus: 0.00029',
   'passer-domesticus: 0.00005',
   'corvus-corax: 0.00009',
   'hirundo-rustica: 0.00049',
   'phoebastria-immutabilis: 0.00005',
   'loxia-curvirostra: 0.00022',
   'sturnus-vulgaris: 0.00023',
   'alauda-arvensis: 0.00008',
   'troglodytes-aedon: 0.00060',
   'melospiza-melodia: 0.00008',
   'luscinia-svecica: 0.00002',
   'agelaius-phoeniceus: 0.00002',
   'mimus-polyglottos: 0.00015',
   'rissa-tridactyla: 0.00002',
   'thryomanes-bewickii: 0.00006',
   'zonotrichia-albicollis: 0.00042',
   'turdus-migratorius: 0.00417',
   'cardinalis-cardinalis: 0.05695',
   'vireo-gilvus: 0.00036',
   'pipilo-maculatus: 0.00007',
   'thryothorus-ludovicianus: 0.00178',
   'cistothorus-palustris: 0.00003',
   'catharus-ustulatus: 0.00038',
   'asio-otus: 0.00016',
   'toxostoma-curvirostre: 0.00001',
   'myiarchus-tyrannulus: 0.00001',
   'catharus-guttatus: 0.00124',
   'vireo-olivaceus: 0.00021',
   'junco-hyemalis: 0.00014',
   'setophaga-ruticill

In [5]:
# Print results
for idx, section in enumerate(result["predictions"]):
    print("==========")
    print(f"Section: {idx}")
    print("==========")
    for pred in section:
        print(pred)

Section: 0
passer-montanus: 0.00029
passer-domesticus: 0.00005
corvus-corax: 0.00009
hirundo-rustica: 0.00049
phoebastria-immutabilis: 0.00005
loxia-curvirostra: 0.00022
sturnus-vulgaris: 0.00023
alauda-arvensis: 0.00008
troglodytes-aedon: 0.00060
melospiza-melodia: 0.00008
luscinia-svecica: 0.00002
agelaius-phoeniceus: 0.00002
mimus-polyglottos: 0.00015
rissa-tridactyla: 0.00002
thryomanes-bewickii: 0.00006
zonotrichia-albicollis: 0.00042
turdus-migratorius: 0.00417
cardinalis-cardinalis: 0.05695
vireo-gilvus: 0.00036
pipilo-maculatus: 0.00007
thryothorus-ludovicianus: 0.00178
cistothorus-palustris: 0.00003
catharus-ustulatus: 0.00038
asio-otus: 0.00016
toxostoma-curvirostre: 0.00001
myiarchus-tyrannulus: 0.00001
catharus-guttatus: 0.00124
vireo-olivaceus: 0.00021
junco-hyemalis: 0.00014
setophaga-ruticilla: 0.00078
dumetella-carolinensis: 0.00022
aegolius-funereus: 0.00002
larus-canus: 0.00000
icteria-virens: 0.00004
morus-bassanus: 0.00000
streptopelia-decaocto: 0.00006
icterus-bull